In [47]:
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras import metrics, Input
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [48]:
METRICS = [metrics.RootMeanSquaredError(name='rms'), metrics.MeanAbsoluteError(name='mae')]

In [74]:
ENCODING_DIM = 16 #Desired Dimension
BATCH_SIZE = 64
EPOCHS = 50

In [75]:
def make_and_train_autoencoder(X_train, metrics=METRICS):
    
    len_input_output = X_train.shape[-1]
    input_ = Input(shape=(len_input_output,))
    encoded = Dense(units=ENCODING_DIM*2, activation="relu")(input_)
    bottleneck = Dense(units=ENCODING_DIM, 
                       activation="relu")(encoded)
    decoded = Dense(units=ENCODING_DIM*2, 
                    activation="relu")(bottleneck)
    output = Dense(units=len_input_output, 
                    activation="linear")(decoded)
    #Training is performed on the entire autoencoder
    autoencoder = Model(inputs=input_, outputs=output)
    autoencoder.compile(optimizer='adam', loss='mean_squared_error',
                        metrics=[metrics])
    autoencoder.fit(X_train, X_train,
                    batch_size=BATCH_SIZE,
                    epochs=EPOCHS)
    #Use only the encoder part for dimensionality reduction
    encoder = Model(inputs=input_, outputs=bottleneck)
    return autoencoder, encoder

In [76]:
df = pd.read_csv('training_ROS.csv')
qw = pd.read_csv('validation_set.csv')

In [77]:
X, y = df.iloc[:,:-1], df.iloc[:, -1]

In [ ]:
auto, encode = make_and_train_autoencoder(X, metrics=METRICS)

Epoch 1/50
512/512 [==============================] - 7s 9ms/step - loss: 0.0074 - rms: 0.0726 - mae: 0.0382
Epoch 2/50
512/512 [==============================] - 4s 7ms/step - loss: 9.1966e-04 - rms: 0.0303 - mae: 0.0174
Epoch 3/50
512/512 [==============================] - 4s 7ms/step - loss: 5.5984e-04 - rms: 0.0237 - mae: 0.0124
Epoch 4/50
512/512 [==============================] - 4s 8ms/step - loss: 3.6246e-04 - rms: 0.0190 - mae: 0.0094
Epoch 5/50
512/512 [==============================] - 3s 7ms/step - loss: 2.6789e-04 - rms: 0.0164 - mae: 0.0076
Epoch 6/50
512/512 [==============================] - 3s 7ms/step - loss: 2.3918e-04 - rms: 0.0155 - mae: 0.0070
Epoch 7/50
512/512 [==============================] - 3s 7ms/step - loss: 2.2582e-04 - rms: 0.0150 - mae: 0.0063
Epoch 8/50
512/512 [==============================] - 3s 7ms/step - loss: 2.1835e-04 - rms: 0.0148 - mae: 0.0059
Epoch 9/50
512/512 [==============================] - 3s 7ms/step - loss: 2.1488e-04 - rms: 0.0147 -

In [59]:
q = encode.predict(X)

In [63]:
q.shape

(32710, 8)

In [68]:
q = np.array(q)
df = pd.DataFrame(data=q,index=None,columns=None)

In [71]:
df['label'] = y
df

,0,1,2,3,4,5,6,7,label
0,0.216690,0.188490,0.0,0.190135,0.354213,0.440773,0.490538,0.325851,0.0
1,0.141760,0.381643,0.0,0.342938,0.267937,0.615193,0.555364,0.535768,1.0
2,0.326538,0.609432,0.0,0.551288,0.413152,0.690596,0.311742,0.166114,0.0
3,0.292853,0.429766,0.0,0.125191,0.273994,0.396629,0.382142,0.333955,0.0
4,0.285552,0.280533,0.0,0.136062,0.256063,0.242172,0.208914,0.263304,0.0
...,...,...,...,...,...,...,...,...,...
32705,0.312745,0.213432,0.0,0.177604,0.211671,0.254998,0.213962,0.247138,1.0
32706,0.248369,0.521588,0.0,0.730797,0.598629,1.044003,0.386442,0.397927,1.0
32707,0.343048,0.708353,0.0,0.751305,0.673825,0.816179,0.666820,0.447209,1.0
32708,0.374286,0.533337,0.0,0.496663,0.505217,0.341860,0.231692,0.067951,1.0


In [72]:
df.to_csv('training_cnn_autoencoded.csv', index=False)

In [73]:
#### CNN autoencode
df = pd.read_csv('training_CNN.csv')
X, y = df.iloc[:,:-1], df.iloc[:, -1]
auto, encode = make_and_train_autoencoder(X, metrics=METRICS)
q = encode.predict(X)
w = np.array(q)
df = pd.DataFrame(data=w,index=None,columns=None)
df['label'] = y
df

Epoch 1/50
512/512 [==============================] - 6s 7ms/step - loss: 0.0137 - rms: 0.0863 - mae: 0.0445
Epoch 2/50
512/512 [==============================] - 3s 7ms/step - loss: 0.0028 - rms: 0.0526 - mae: 0.0318
Epoch 3/50
512/512 [==============================] - 3s 7ms/step - loss: 0.0020 - rms: 0.0447 - mae: 0.0260
Epoch 4/50
512/512 [==============================] - 3s 7ms/step - loss: 0.0018 - rms: 0.0429 - mae: 0.0234
Epoch 5/50
304/512 [================>.............] - ETA: 1s - loss: 0.0018 - rms: 0.0426 - mae: 0.0228

KeyboardInterrupt: 